In [4]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from numpy import array

In [5]:
data = pd.read_csv('city_hour.csv')

In [6]:
data.head()

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01 01:00:00,NaN,NaN,1.00,40.01,36.37,NaN,1.00,122.07,NaN,0.0,0.0,0.0,NaN,NaN
1,Ahmedabad,2015-01-01 02:00:00,NaN,NaN,0.02,27.75,19.73,NaN,0.02,85.90,NaN,0.0,0.0,0.0,NaN,NaN
2,Ahmedabad,2015-01-01 03:00:00,NaN,NaN,0.08,19.32,11.08,NaN,0.08,52.83,NaN,0.0,0.0,0.0,NaN,NaN
3,Ahmedabad,2015-01-01 04:00:00,NaN,NaN,0.30,16.45,9.20,NaN,0.30,39.53,153.58,0.0,0.0,0.0,NaN,NaN
4,Ahmedabad,2015-01-01 05:00:00,NaN,NaN,0.12,14.90,7.85,NaN,0.12,32.63,NaN,0.0,0.0,0.0,NaN,NaN


In [7]:
# Convert 'Datetime' to datetime format and set it as index
data['Datetime'] = pd.to_datetime(data['Datetime'])
data = data.set_index('Datetime')

In [8]:

# Drop non-numerical columns
data = data.drop(['City', 'AQI_Bucket'], axis=1)
data = data.dropna()

In [9]:
data.dropna().count()

PM2.5      129277
PM10       129277
NO         129277
NO2        129277
NOx        129277
NH3        129277
CO         129277
SO2        129277
O3         129277
Benzene    129277
Toluene    129277
Xylene     129277
AQI        129277
dtype: int64

In [10]:
data.head()

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI
Datetime,,,,,,,,,,,,,
2017-11-25 09:00:00,104.00,148.50,1.93,23.00,13.75,9.80,0.1,15.30,117.62,0.30,10.40,0.23,155.0
2017-11-25 10:00:00,94.50,142.00,1.33,16.25,9.75,9.65,0.1,17.00,136.23,0.28,7.10,0.15,159.0
2017-11-25 11:00:00,82.75,126.50,1.47,14.83,9.07,9.70,0.1,15.40,149.92,0.20,4.55,0.08,173.0
2017-11-25 14:00:00,68.50,117.00,1.35,13.60,8.35,7.40,0.1,21.80,161.70,0.10,2.30,0.00,191.0
2017-11-25 15:00:00,69.25,112.25,1.52,11.80,7.55,9.25,0.1,21.38,161.68,0.10,2.35,0.00,191.0


In [11]:

# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)


In [12]:

# Define the sequence split function
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [13]:
# Split the sequence
n_steps = 3
X, y = split_sequence(data.values, n_steps)

In [14]:

# Reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = data.shape[1]
X = X.reshape((X.shape[0], X.shape[1], n_features))


In [15]:

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')


In [15]:

# Train the model
model.fit(X_train, y_train, epochs=200, verbose=1)


Epoch 1/200
17697/17697 [==============================] - 53s 3ms/step - loss: nan
Epoch 2/200
13128/17697 [=====================>........] - ETA: 13s - loss: nan


KeyboardInterrupt



In [16]:
# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', loss)

808/808 [==============================] - 2s 2ms/step - loss: nan
Test loss: nan


In [17]:
# accuracy
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

808/808 [==============================] - 1s 1ms/step


/Users/mahas/PycharmProjects/pragyan-backend/.venv/lib/python3.9/site-packages/sklearn/utils/_array_api.py:290: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)
/Users/mahas/PycharmProjects/pragyan-backend/.venv/lib/python3.9/site-packages/sklearn/utils/_array_api.py:290: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y_pred contains NaN.

In [1]:
import pickle
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

2024-01-28 00:09:35.865374: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/mahas/PycharmProjects/pragyan-backend/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [22]:
print(y_test)
indices = [i for i in range(len(y_test)) if 0 in y_test[i]]

[[0.02919117 0.08104104 0.04383353 ... 0.01863166 0.00767322 0.10916442]
 [0.02298092 0.03561562 0.01100603 ... 0.02120677 0.01328498 0.16576819]
 [0.07529301 0.09685686 0.04018867 ... 0.01151224 0.00763504 0.06199461]
 ...
 [0.05043202 0.09997998 0.04280916 ... 0.09300682 0.01439206 0.11455526]
 [0.02897116 0.04704705 0.01808133 ... 0.04594799 0.00992556 0.11859838]
 [0.06334253 0.09083083 0.06036639 ... 0.00126231 0.00259592 0.11590296]]


In [18]:
y_test = [x for x in y_test if x]

ValueError: Input contains NaN.